In [6]:
import json

In [12]:
with open('./performanceOutput.json') as f:
    raw = json.load(f)

In [13]:
len(raw)

587

In [25]:
raw[300]

{'metrics': {}, 'time': 1522170688301}

In [30]:
[x for x in profile['nodes'] if 'children' in x and 18 in x['children']]

[{'callFrame': {'columnNumber': 14554,
   'functionName': '',
   'lineNumber': 0,
   'scriptId': '19',
   'url': ''},
  'children': [18],
  'hitCount': 3,
  'id': 17,
  'positionTicks': [{'line': 1, 'ticks': 3}]}]

In [25]:
import numpy as np

# Unit: microseconds
np.average(profile['timeDeltas']), np.std(profile['timeDeltas'])

(1277.927694235589, 407.15091960880113)

In [21]:
from collections import Counter
c = Counter()
for x in profile['samples']:
    c[x] += 1
max(c.items(), key=lambda x: x[1])

(3, 7816)

In [14]:
sorted(profile['nodes'], key=lambda x: x['hitCount'], reverse=True)[:10]

[{'callFrame': {'columnNumber': -1,
   'functionName': '(idle)',
   'lineNumber': -1,
   'scriptId': '0',
   'url': ''},
  'hitCount': 7816,
  'id': 3},
 {'callFrame': {'columnNumber': -1,
   'functionName': '(program)',
   'lineNumber': -1,
   'scriptId': '0',
   'url': ''},
  'hitCount': 103,
  'id': 2},
 {'callFrame': {'columnNumber': 19484,
   'functionName': 'h.(anonymous function).(anonymous function)',
   'lineNumber': 0,
   'scriptId': '19',
   'url': ''},
  'hitCount': 7,
  'id': 19,
  'positionTicks': [{'line': 1, 'ticks': 7}]},
 {'callFrame': {'columnNumber': 25413,
   'functionName': '_0x2d0c.lbeIEt',
   'lineNumber': 0,
   'scriptId': '16',
   'url': 'https://www.cryptonoter.me/prototyper-show.js'},
  'children': [8, 10, 11],
  'hitCount': 6,
  'id': 7,
  'positionTicks': [{'line': 1, 'ticks': 6}]},
 {'callFrame': {'columnNumber': -1,
   'functionName': '(garbage collector)',
   'lineNumber': -1,
   'scriptId': '0',
   'url': ''},
  'hitCount': 4,
  'id': 9},
 {'callFrame'